In [6]:
import numpy as np
import cv2
import os
import csv

def create_gist_descriptor(image, orientations=(8, 8, 8, 8), nblocks=4):
    """
    Compute the GIST descriptor for an image.

    Parameters:
        image (numpy.ndarray): Input image.
        orientations (tuple): Number of orientations for each scale.
        nblocks (int): Number of blocks for dividing the image.

    Returns:
        numpy.ndarray: GIST descriptor.
    """
    # Convert to grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Parameters
    nscales = len(orientations)
    max_freq = 0.25
    min_freq = 0.02
    gist = []

    # Compute the Gabor filters
    for scale in range(nscales):
        for orientation in range(orientations[scale]):
            freq = max_freq / (2 ** scale)
            theta = orientation * np.pi / orientations[scale]
            lambd = 1.0 / freq
            sigma = 0.56 * lambd
            gamma = 0.5
            psi = 0
            kernel = cv2.getGaborKernel((nblocks, nblocks), sigma, theta, lambd, gamma, psi)
            filtered = cv2.filter2D(image, cv2.CV_32F, kernel)
            gist.append(filtered)

    # Divide the image into blocks and compute histograms
    descriptor = []
    height, width = image.shape
    block_height, block_width = height // nblocks, width // nblocks

    for y in range(nblocks):
        for x in range(nblocks):
            for filtered in gist:
                block = filtered[y*block_height:(y+1)*block_height, x*block_width:(x+1)*block_width]
                descriptor.append(block.mean())

    return np.array(descriptor)

def process_images_in_folder(folder_path, output_csv):
    results = []

    # Determine label based on folder name pattern
    label = 0 if "Negative_" in os.path.basename(folder_path) else 1

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            file_path = os.path.join(folder_path, filename)

            # Read the image
            image = cv2.imread(file_path)
            if image is None:
                print(f"Failed to read image: {file_path}")
                continue

            # Extract GIST features
            gist_features = create_gist_descriptor(image)

            # Flatten the GIST features
            flattened_features = gist_features.flatten()

            # Append the results
            results.append({
                'GistFeature': flattened_features,
                'label': label
            })

    # Write the results to a CSV file
    with open(output_csv, mode='w', newline='') as file:
        fieldnames = ['GistFeature', 'label']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for result in results:
            writer.writerow({
                'GistFeature': ','.join(map(str, result['GistFeature'])),
                'label': result['label']
            })

def process_multiple_folders(base_folder):
    # Iterate over all subfolders in the base folder
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            output_csv = f'{folder_name}_GIST_features.csv'
            process_images_in_folder(folder_path, output_csv)
            print(f"GIST features have been calculated for images in {folder_path} and written to {output_csv}")

folders = ['Rice_photos/BC-15', 'Rice_photos/Huongthom', 'Rice_photos/Nep87', 'Rice_photos/Q5', 'Rice_photos/Thien_uu', 'Rice_photos/Xi23']
for folder in folders:
    process_multiple_folders(folder)


GIST features have been calculated for images in Rice_photos/BC-15\BC-15 and written to BC-15_GIST_features.csv
GIST features have been calculated for images in Rice_photos/BC-15\Negative_BC-15 and written to Negative_BC-15_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Huongthom\Huong_thom-1 and written to Huong_thom-1_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Huongthom\Negative_Huong_thom-1 and written to Negative_Huong_thom-1_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Nep87\Negative_Nep-87 and written to Negative_Nep-87_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Nep87\Nep-87 and written to Nep-87_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Q5\Negative_Q-5_modify and written to Negative_Q-5_modify_GIST_features.csv
GIST features have been calculated for images in Rice_photos/Q5\Q-5_modify and written to Q-5_mod